<a href="https://colab.research.google.com/github/alvinsenjaya/TomatoPlantDiseases/blob/master/Optimize_Saved_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Required Libraries**

In [0]:
import os
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from google.colab import files

# **Upload Saved Model**

In [0]:
uploaded = files.upload()
!unzip /content/improved_cnn_inceptionv3.zip

# **Upload Saved Model From Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [4]:
!cp '/content/gdrive/My Drive/Saved Model/improved_cnn_inceptionv3.zip' /content/
!unzip /content/improved_cnn_inceptionv3.zip

Archive:  /content/improved_cnn_inceptionv3.zip
   creating: content/improved_cnn_inceptionv3/
  inflating: content/improved_cnn_inceptionv3/saved_model.pb  
   creating: content/improved_cnn_inceptionv3/assets/
   creating: content/improved_cnn_inceptionv3/variables/
  inflating: content/improved_cnn_inceptionv3/variables/variables.index  
  inflating: content/improved_cnn_inceptionv3/variables/variables.data-00000-of-00002  
  inflating: content/improved_cnn_inceptionv3/variables/variables.data-00001-of-00002  


In [5]:
model = tf.keras.models.load_model('/content/content/improved_cnn_inceptionv3/')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 6, 6, 2048)        21802784  
_________________________________________________________________
dropout_6 (Dropout)          (None, 6, 6, 2048)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 73728)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              75498496  
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                10250     
Total params: 97,311,530
Trainable params: 97,277,098
Non-trainable params: 34,432
_____________________________________

# **Convert to Tensorflow Lite**

In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/content/improved_cnn_inceptionv3/')
tflite_model = converter.convert()
open("improved_cnn_inceptionv3.tflite", "wb").write(tflite_model)

389142456

# **Optimize Model using Tensorflow Lite**

In [7]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/content/improved_cnn_inceptionv3/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
open("improved_cnn_inceptionv3_quantized.tflite", "wb").write(tflite_quant_model)

97372032

# **Evaluate Model**

In [0]:
os.environ['KAGGLE_USERNAME']="xxxxx"
os.environ['KAGGLE_KEY']="xxxxxxxxxxxxxxxx"

In [12]:
!kaggle datasets download noulam/tomato

 98% 729M/743M [00:12<00:00, 70.4MB/s]
100% 743M/743M [00:12<00:00, 61.2MB/s]


In [0]:
!unzip tomato.zip

In [15]:
VALIDATION_DIR = '/content/New Plant Diseases Dataset(Augmented)/valid/'
validation_datagen = ImageDataGenerator(rescale = 1./255)


validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(256,256),
    class_mode='categorical',
    batch_size=128
)

Found 4585 images belonging to 10 classes.


In [18]:
model.evaluate(validation_generator)

36/36 [==============================] - 13s 352ms/step - loss: 0.1318 - accuracy: 0.9616


[0.13177409768104553, 0.9616139531135559]

In [19]:
tflite_interpreter = tf.lite.Interpreter(model_path='/content/improved_cnn_inceptionv3_quantized.tflite')

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 256 256   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [ 1 10]
type: <class 'numpy.float32'>


In [21]:
tflite_interpreter = tf.lite.Interpreter(model_path='/content/improved_cnn_inceptionv3.tflite')

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 256 256   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [ 1 10]
type: <class 'numpy.float32'>


# **Download Saved Model**

In [0]:
files.download("/content/improved_cnn_inceptionv3.tflite")

In [0]:
files.download("/content/improved_cnn_inceptionv3_quantized.tflite")

# **Save to Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
!cp '/content/improved_cnn_inceptionv3.tflite' '/content/gdrive/My Drive/Saved Model'

In [0]:
!cp '/content/improved_cnn_inceptionv3_quantized.tflite' '/content/gdrive/My Drive/Saved Model'